# Project - New Employee Onboarding Assistant

A friendly HR assistant that helps new employees get started — explains policies, checks training schedules, finds contacts, and shows office images — while speaking replies and displaying visuals.

In [2]:
# imports

import os, json, sqlite3, base64
import json
from dotenv import load_dotenv
import gradio as gr
from io import BytesIO
from PIL import Image
import sys
sys.path.append(os.path.abspath(os.path.join("..", ".."))) 
from openai import OpenAI


In [3]:
# Initialization

conn = sqlite3.connect("onboarding.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS employees (
    name TEXT,
    role TEXT,
    start_date TEXT,
    manager TEXT,
    location TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS training (
    role TEXT,
    course TEXT,
    duration TEXT
)
""")

cursor.executemany("INSERT INTO employees VALUES (?, ?, ?, ?, ?)", [
    ("Alice", "DevOps Engineer", "2025-10-15", "Bharat Puri", "Pune HQ"),
    ("Ravi", "Data Analyst", "2025-10-20", "Neha Kapoor", "Bangalore"),
])

cursor.executemany("INSERT INTO training VALUES (?, ?, ?)", [
    ("DevOps Engineer", "Cloud Infrastructure Basics", "2 weeks"),
    ("DevOps Engineer", "Security and Compliance", "1 week"),
    ("Data Analyst", "Python for Data Analysis", "3 weeks")
])

conn.commit()
conn.close()

In [5]:
load_dotenv(override=True)

openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print(f"✅ API Key loaded: {openai_api_key[:8]}****")
else:
    print("❌ OPENAI_API_KEY not set")

MODEL = "gpt-4.1-mini"
openai = OpenAI()
DB = "onboarding.db"

✅ API Key loaded: sk-proj-****


In [6]:
system_message = """
You are WelcomeAI, an onboarding assistant for new employees.
Be friendly and concise (1–2 sentences). 
Always be accurate and supportive. If unsure, say so politely.
"""

In [7]:
# -------------------- TOOLS --------------------

def get_employee_info(name):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM employees WHERE lower(name)=?", (name.lower(),))
        result = cursor.fetchone()
        if result:
            name, role, start_date, manager, location = result
            return f"{name} is joining as a {role} on {start_date}. Manager: {manager}. Location: {location}."
        else:
            return "I couldn’t find that employee in the database."

In [8]:
def get_training_schedule(role):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT course, duration FROM training WHERE role=?", (role,))
        results = cursor.fetchall()
        if results:
            schedule = "; ".join([f"{course} ({duration})" for course, duration in results])
            return f"Training schedule for {role}: {schedule}"
        else:
            return "No training schedule found for that role."

In [9]:
# Tool schema definitions
employee_tool = {
    "name": "get_employee_info",
    "description": "Retrieve onboarding information about a new employee.",
    "parameters": {
        "type": "object",
        "properties": {
            "employee_name": {"type": "string", "description": "The full name of the employee."}
        },
        "required": ["employee_name"],
    },
}

In [10]:
training_tool = {
    "name": "get_training_schedule",
    "description": "Get the training schedule for a given role.",
    "parameters": {
        "type": "object",
        "properties": {
            "role": {"type": "string", "description": "The job role of the employee."}
        },
        "required": ["role"],
    },
}

In [11]:

tools = [{"type": "function", "function": employee_tool},
         {"type": "function", "function": training_tool}]

In [14]:
# -------------------- MULTI-MODAL --------------------
def artist(topic):
    prompt = f"A friendly HR welcome image showing {topic}, office vibes, smiling team, pop-art style"
    image_response = openai.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        response_format="b64_json"
    )
    img_base64 = image_response.data[0].b64_json
    img_data = base64.b64decode(img_base64)
    return Image.open(BytesIO(img_data))

def talker(message):
    response = openai.audio.speech.create(
        model="gpt-4o-mini-tts",
        voice="alloy",
        input=message
    )
    return response.content

In [15]:
# -------------------- AGENT LOGIC --------------------

def handle_tool_calls(message):
    responses, topics = [], []
    for call in message.tool_calls:
        if call.function.name == "get_employee_info":
            args = json.loads(call.function.arguments)
            name = args.get("employee_name")
            topics.append(name)
            info = get_employee_info(name)
            responses.append({"role": "tool", "content": info, "tool_call_id": call.id})
        elif call.function.name == "get_training_schedule":
            args = json.loads(call.function.arguments)
            role = args.get("role")
            topics.append(role)
            info = get_training_schedule(role)
            responses.append({"role": "tool", "content": info, "tool_call_id": call.id})
    return responses, topics



In [16]:
def chat(history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    topics, image = [], None

    while response.choices[0].finish_reason == "tool_calls":
        msg = response.choices[0].message
        responses, topics = handle_tool_calls(msg)
        messages.append(msg)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    reply = response.choices[0].message.content
    voice = talker(reply)

    if topics:
        image = artist(topics[0])

    return history + [{"role": "assistant", "content": reply}], voice, image

In [17]:
# -------------------- GRADIO UI --------------------

def put_message_in_chatbot(message, history):
    return "", history + [{"role": "user", "content": message}]

with gr.Blocks() as ui:
    gr.Markdown("## 🧑‍💼 WelcomeAI — Your HR Onboarding Companion")
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500, interactive=False)
    with gr.Row():
        audio_output = gr.Audio(autoplay=True)
    with gr.Row():
        message = gr.Textbox(label="Ask me about onboarding, training, or company info:")

    message.submit(put_message_in_chatbot, [message, chatbot], [message, chatbot]).then(
        chat, chatbot, [chatbot, audio_output, image_output]
    )

In [19]:
ui.launch(inbrowser=True, auth=("hradmin", "welcome123"))

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
* To create a public link, set `share=True` in `launch()`.
